<a href="https://colab.research.google.com/github/m-evdokimov/dls-detector/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print('i am here')

i am here
